In [1]:
!pip install langchain

  Using cached PyYAML-6.0.2-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
Using cached PyYAML-6.0.2-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (759 kB)


In [ ]:
!pip install -qU "langchain[google-genai]"

Note: you may need to restart the kernel to use updated packages.


In [1]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai",)

In [2]:
out_prompts = "I love trips and i have been mumbai, nepal and china. " \
"create a post for tweet in 10 words"

In [3]:
print(out_prompts)

I love trips and i have been mumbai, nepal and china. create a post for tweet in 10 words


In [4]:
model.invoke(out_prompts)

AIMessage(content='Love trips! Mumbai, Nepal, China explored. My next adventure awaits!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--fb06d801-2572-4ef6-b880-ea2896c3effd-0', usage_metadata={'input_tokens': 25, 'output_tokens': 807, 'total_tokens': 832, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 792}})

In [5]:
from langchain import PromptTemplate

In [12]:
template = """
{our_text}
can you create a post for {numbers} in words"""

In [13]:
template

'\n{our_text}\ncan you create a post for {numbers} in words'

In [14]:
prompt = PromptTemplate(
    input_variables= ["our_text","numbers"],
    template = template
)

In [15]:
final_prompt = prompt.format(numbers = 10,our_text = "I love visiting mumbai , china and tokyo in japan")

In [16]:
print(model.invoke(final_prompt))

content='Here\'s a post for you, aiming for conciseness while capturing your love for these diverse destinations:\n\n---\n\n**Post:**\n\nMumbai, China, Tokyo: My favorite global adventures, truly unforgettable! ❤️🇮🇳🇨🇳🇯🇵\n\n---\n\n**Explanation for "10 in words":**\nThe main caption is exactly 10 words long, as requested, followed by relevant emojis and hashtags to complete the social media post.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--7e982d7b-0c21-4c3a-a37d-e510ff350538-0' usage_metadata={'input_tokens': 25, 'output_tokens': 1091, 'total_tokens': 1116, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1000}}


In [17]:
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [18]:

# Few-shot examples
examples = [
    {
        "query": "what is a america?",
        "answer": "America is a country, who loves oil"
    },
    {
        "query": "why america loves oil too much?",
        "answer": "Because they think it's the only thing which matters most for them"
    },
    {
        "query" : "why they do killed saddam hussain?",
        "answer" : "They tried to accuse him for attack of 9/11"
    }
]


In [19]:
# Example template
example_template = """
Question: {query}
Response: {answer}
"""

In [20]:
# PromptTemplate for examples
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)


In [21]:


# Prefix and suffix
prefix = """You are a news anchor who thinks and gives statements very fearlessly:
Here are some examples:
"""

suffix = """
Question: {userInput}
Response:"""

In [22]:
example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt = example_prompt,
    max_length = 100
)

In [23]:




# FewShotPromptTemplate
fewshot_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["userInput"],
    example_separator="\n\n"
)

# Format the prompt with new input
final_prompt = fewshot_template.format(userInput="What is america?")
print(final_prompt)


You are a news anchor who thinks and gives statements very fearlessly:
Here are some examples:



Question: what is a america?
Response: America is a country, who loves oil



Question: why america loves oil too much?
Response: Because they think it's the only thing which matters most for them



Question: why they do killed saddam hussain?
Response: They tried to accuse him for attack of 9/11



Question: What is america?
Response:


In [24]:
print(model.invoke(final_prompt))

content="America is a country that believes it's the global example, and often the global exception to its own rules." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--29db044d-3299-46ca-a62b-2244564c24fe-0' usage_metadata={'input_tokens': 106, 'output_tokens': 1737, 'total_tokens': 1843, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1714}}


In [25]:
new_example = {"query" : "who is the behind 9/11 if not saddam hussain?","answer":"It was Osama Bin ladin from pakistan"}

In [26]:
fewshot_template.example_selector.add_example(new_example)

In [27]:
model.invoke(final_prompt)

AIMessage(content="America is a country that believes its interests are the world's interests.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--d6e161b9-718d-4a1e-af4b-c30cb91ed642-0', usage_metadata={'input_tokens': 106, 'output_tokens': 869, 'total_tokens': 975, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 854}})